In [1]:
import os

import pickle
import time
from langchain.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader,TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env


True

In [2]:
file_path = "faiss_store_deepseek.pkl"
llm = ChatOllama(model="deepseek-r1:7b", base_url="http://127.0.0.1:11500")

/tmp/ipykernel_125570/4227689947.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="deepseek-r1:7b", base_url="http://127.0.0.1:11500")


In [3]:
loader=TextLoader("/home/jaskaran/Desktop/langchain-main/2_news_research_tool_project/notebooks/nvda_news_1.txt")
data=loader.load()
data

[Document(metadata={'source': '/home/jaskaran/Desktop/langchain-main/2_news_research_tool_project/notebooks/nvda_news_1.txt'}, page_content="The stock of NVIDIA Corp (NASDAQ:NVDA) experienced a daily loss of -3.56% and a 3-month gain of 32.35%. With an Earnings Per Share (EPS) (EPS) of $1.92, the question arises: is the stock significantly overvalued? This article aims to provide a detailed valuation analysis of NVIDIA, offering insights into its financial strength, profitability, growth, and more. We invite you to delve into this comprehensive analysis.\n\nCompany Overview\nWarning! GuruFocus has detected 10 Warning Signs with NVDA. Click here to check it out.\n\nNVDA 30-Year Financial Data\n\nThe intrinsic value of NVDA\n\n\nNVIDIA Corp (NASDAQ:NVDA) is a leading designer of discrete graphics processing units that enhance the experience on computing platforms. The firm's chips are widely used in various end markets, including PC gaming and data centers. In recent years, NVIDIA has br

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000
    )
docs = text_splitter.split_documents(data)
docs

[Document(metadata={'source': '/home/jaskaran/Desktop/langchain-main/2_news_research_tool_project/notebooks/nvda_news_1.txt'}, page_content='The stock of NVIDIA Corp (NASDAQ:NVDA) experienced a daily loss of -3.56% and a 3-month gain of 32.35%. With an Earnings Per Share (EPS) (EPS) of $1.92, the question arises: is the stock significantly overvalued? This article aims to provide a detailed valuation analysis of NVIDIA, offering insights into its financial strength, profitability, growth, and more. We invite you to delve into this comprehensive analysis.\n\nCompany Overview\nWarning! GuruFocus has detected 10 Warning Signs with NVDA. Click here to check it out.\n\nNVDA 30-Year Financial Data\n\nThe intrinsic value of NVDA'),
 Document(metadata={'source': '/home/jaskaran/Desktop/langchain-main/2_news_research_tool_project/notebooks/nvda_news_1.txt'}, page_content="Company Overview\nWarning! GuruFocus has detected 10 Warning Signs with NVDA. Click here to check it out.\n\nNVDA 30-Year Fi

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)

/tmp/ipykernel_125570/3169752069.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/jaskaran/anaconda3/envs/res/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jaskaran/anaconda3/envs/res/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possi

In [6]:
with open(file_path, "wb") as f:
    pickle.dump(vectorstore, f)

In [7]:
query="is NVIDIA overvalued? "

In [8]:
with open(file_path, "rb") as f:
    vectorstore = pickle.load(f)

In [20]:
chain = RetrievalQA.from_llm(llm=llm, retriever=vectorstore.as_retriever())
result = chain({"query": query}, return_only_outputs=True)

In [21]:
result

{'result': "<think>\nOkay, so I need to figure out whether NVIDIA is overvalued based on the provided context. Let me start by going through each piece of information step by step.\n\nFirst, there's a mention of the GF Value. The GF Value acts as an intrinsic value calculated using historical trading multiples, a GuruFocus adjustment factor, and future business performance estimates. If the stock price is above this line significantly, it's overvalued; if below, undervalued.\n\nThe context says that NVIDIA's current price is $418.01, which is higher than their GF Value of $310.28. That already points towards being overvalued. Additionally, the market cap is stated as $1 trillion, which seems high compared to its intrinsic value based on GF.\n\nLooking at the financial strength, NVIDIA has a cash-to-debt ratio of 1.27, which is worse than 58% of companies in the semiconductor industry. However, their overall financial strength rating is an 8 out of 10, indicating they're in good shape b

In [22]:
import re

In [23]:
def extract_outside_think_tags(text):
    # Remove everything inside <think>...</think> including the tags
    cleaned_text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    return cleaned_text.strip()

In [24]:

result_new = result['result']

result_new

"<think>\nOkay, so I need to figure out whether NVIDIA is overvalued based on the provided context. Let me start by going through each piece of information step by step.\n\nFirst, there's a mention of the GF Value. The GF Value acts as an intrinsic value calculated using historical trading multiples, a GuruFocus adjustment factor, and future business performance estimates. If the stock price is above this line significantly, it's overvalued; if below, undervalued.\n\nThe context says that NVIDIA's current price is $418.01, which is higher than their GF Value of $310.28. That already points towards being overvalued. Additionally, the market cap is stated as $1 trillion, which seems high compared to its intrinsic value based on GF.\n\nLooking at the financial strength, NVIDIA has a cash-to-debt ratio of 1.27, which is worse than 58% of companies in the semiconductor industry. However, their overall financial strength rating is an 8 out of 10, indicating they're in good shape but not top-

In [25]:

outside_text = extract_outside_think_tags(result_new)

print(outside_text)    

Based on the analysis of NVIDIA's intrinsic value through the GF Value calculation, along with its stock price and market capitalization, it is evident that NVIDIA's stock is significantly overvalued. The current stock price of $418.01 exceeds the calculated intrinsic value ($310.28), and a market cap of $1 trillion further supports this conclusion. Despite potential future growth in product expansion into AI and autonomous driving, the significant gap between the stock's price and its intrinsic value suggests that NVIDIA is overvalued at present. 

**Answer:** Yes, NVIDIA's stock appears to be significantly overvalued based on its current price compared to its GF Value and market capitalization.
